<a href="https://colab.research.google.com/github/jespimentel/leitor_planilha_vivo/blob/master/leitor_de_planilha_vivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leitor de Planilhas da Vivo
https://github.com/jespimentel

OBJETIVO

1. A Vivo fornece em planilha Excel, mediante requisição judicial, os metadados das chamadas realizadas a partir de telefones celulares.

2. Na planilha da operadora é possível identificar as ERBs utilizadas nas chamadas pelas colunas comuns de duas diferentes pastas de trabalho.

3. Este script auxilia o investigador na tarefa de fazer esse relacionamento. Além disso, converte as coordenadas geográficas em formato decimal para plotagem.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
path_da_planilha = '/content/consulta_16996222XXX_id8610473_1320.xlsx'
alvo = '16996222XXX'

chamadas = pd.read_excel (path_da_planilha, sheet_name=0)
erbs = pd.read_excel (path_da_planilha, header=4, sheet_name=2)

In [ ]:
chamadas.columns = chamadas.iloc[4]
chamadas = chamadas[5:]
chamadas.reset_index(drop=True, inplace=True)
chamadas.columns.name = None
chamadas = chamadas [['Data', 'Hora', 'Chamador', 'Chamado', 'Local Origem', 'Local Destino']]
chamadas

In [ ]:
chamadas['Chamador'].value_counts().head(10)

In [ ]:
chamadas['Chamado'].value_counts().head(10)

In [ ]:
erbs.columns = erbs.iloc[0]
erbs = erbs[1:]
erbs = erbs.reset_index(drop=True)
erbs = erbs.dropna(subset=['CGI'])
erbs = erbs[erbs['CGI'].str.strip() != '']
erbs.reset_index(drop=True, inplace=True)
erbs.columns.name = None
erbs = erbs [['CGI', 'UF', 'Cidade', 'Bairro', 'Endereço', 'Azi', 'Latitude', 'Longitude']]
erbs = erbs.dropna(subset=['Latitude'])
erbs = erbs[erbs['Latitude'].str.strip() != '']
erbs

In [ ]:
def converte_dms_dd(coordenada):
  if coordenada[0] == '-':
    new_coord = coordenada[0]
  else:
    new_coord = ''
  coordenada = coordenada.replace(',', '.').split('-') # Ex. de resultado: ['', '22', '47', '35.5']
  if coordenada[0] == '':
    del (coordenada[0])
  graus = int(coordenada[0])
  decimais = float(coordenada[1])/60 + float(coordenada[2])/3600
  new_coord = new_coord + str(graus + decimais)
  return float(new_coord)

# Substituição das coordenadas convertidas em DD no Dataframe
erbs.Latitude = erbs.Latitude.map(lambda x: converte_dms_dd(x))
erbs.Longitude = erbs.Longitude.map(lambda x: converte_dms_dd(x))
erbs

In [ ]:
chamadas_filtered = chamadas[chamadas['Chamador'] == alvo]

# Perform an inner join on 'Local de Origem' and 'CGI'
df_origem = pd.merge(chamadas_filtered, erbs, left_on='Local Origem', right_on='CGI', how='inner')

# Drop rows with missing values in both dataframes
df_origem.dropna(subset=['Local Origem', 'CGI'], inplace=True)

# Display the merged dataframe
df_origem

In [ ]:
chamadas_filtered = chamadas[chamadas['Chamado'] == alvo]

# Perform an inner join on 'Local de Origem' and 'CGI'
df_destino = pd.merge(chamadas_filtered, erbs, left_on='Local Destino', right_on='CGI', how='inner')

# Drop rows with missing values in both dataframes
df_destino.dropna(subset=['Local Destino', 'CGI'], inplace=True)

# Display the merged dataframe
df_destino

In [ ]:
# prompt: concatene df_origem e df_destino. Ordene pelas colunas de Data e Hora.

# Concatenar df_origem e df_destino
df_concat = pd.concat([df_origem, df_destino], ignore_index=True)

# Remove whitespace from 'Azi' column and convert to numeric
df_concat['Azi'] = pd.to_numeric(df_concat['Azi'].str.strip(), errors='coerce')

# Convert to integer (NaN values will remain)
df_concat['Azi'] = df_concat['Azi'].astype('Int64')

# Exibir o DataFrame resultante
display(df_concat)

In [ ]:
# Salva o DataFrame em Excel
df_concat.to_excel(f'erbs_origem_e_destino_{alvo}.xlsx', index=False)

In [ ]:
import folium

def plot_map(df, alvo):
    """
    Plots a map using folium based on the coordinates in the dataframe.

    Args:
    df (DataFrame): DataFrame containing at least 'Latitude', 'Longitude', 'Data', 'Hora', 'Chamador', and 'Chamado' columns.
    target (str): The name to save the map as an HTML file.

    Returns:
    folium.Map: A folium map object centered on the mean coordinates and with markers for each row in the dataframe.
    """
    # Calculate the mean point of the coordinates
    latitude_mean = df['Latitude'].mean()
    longitude_mean = df['Longitude'].mean()

    # Create a map centered at the mean point
    map_ = folium.Map(location=[latitude_mean, longitude_mean], zoom_start=10)

    # Iterate over the DataFrame rows and add markers to the map
    for index, row in df.iterrows():
        # Extract information from the current row
        latitude = row['Latitude']
        longitude = row['Longitude']
        data = row['Data']
        hour = row['Hora']
        caller = row['Chamador']
        call = row['Chamado']

        # Create the popup with information
        popup_text = f"<b>Data:</b> {data}<br><b>Hora:</b> {hour}<br><b>Chamador:</b> {caller}<br><b>Chamado:</b> {call}"

        # Add a marker to the map
        folium.Marker(
            location=[latitude, longitude],
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(map_)

    # Save the map to an HTML file
    map_.save(f'mapa_{alvo}.html')

    return map_

In [ ]:
plot_map(df_concat, alvo)

# Análise simultânea

In [ ]:
df_1 = pd.read_excel ('/content/erbs_origem_e_destino_16996222XXX.xlsx', sheet_name=0)
df_2 = pd.read_excel ('/content/erbs_origem_e_destino_16999756XXX.xlsx', sheet_name=0)

In [ ]:
# prompt: concatene df_1 e df_2.

# Concatenate df_1 and df_2
df_concat_total = pd.concat([df_1, df_2], ignore_index=True)

# Salva o DataFrame em Excel
df_concat_total.to_excel(f'erbs_origem_e_destino_todos.xlsx', index=False)

# Display the concatenated DataFrame
display(df_concat_total.head(5))

In [ ]:
df_concat_total['Chamador'].value_counts().head(10)

In [ ]:
df_concat_total['Chamado'].value_counts().head(10)

In [ ]:
plot_map(df_concat_total, alvo)